<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data : https://data.stackexchange.com/stackoverflow/query/edit/1086274
    - Source : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse
    - Entrainement
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 06/08/2019 

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I - Data</p>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I.1 - Source</p>

Revoir les explications

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I.2 - Preparation des données</p>

In [1]:
import sys
import os
sys.path.insert(0, './Python/')




In [2]:
from tagText import TagText

In [3]:
tagText = TagText()
print(tagText.urlDirectory)
print(tagText.fileName)
data_question = tagText.read_source()
data_question.head()

Data/
QuestionVsTags.csv


,body,title,tags,SelectedTags,Name
0,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo>,"git,version-control,git-commit,",Question
2,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><git-branch><git-remote>,"git,git-branch,",Question
3,<blockquote>\n <p><strong>Moderator Note:</st...,What is the difference between 'git pull' and ...,<git><git-pull><git-fetch>,"git,",Question
4,"<p>I've been messing around with <a href=""http...",What is the correct JSON content type?,<json><http-headers><content-type>,"json,http-headers,content-type,",Question


In [4]:
print(tagText.stopwords)
#print(tagText.badwords)

{'t', 'been', 'theirs', 'into', 'should', "doesn't", 'hasn', 'won', 'further', 'weren', 'because', "mustn't", 'being', 'having', 've', 'for', 'and', 'couldn', 'am', 'll', "needn't", 'does', 'what', 'out', 'can', "haven't", "wouldn't", 'shouldn', 'your', 'will', 'we', 'himself', 'same', "mightn't", 'by', 'as', 'when', 'to', 'before', 's', 'if', "couldn't", "isn't", 'myself', 'which', 'during', 'm', 'shan', 'but', 'other', 'at', 'who', 're', 'her', 'did', 'that', 'ours', 'ain', 'had', 'has', 'then', 'our', 'both', 'why', 'a', 'most', 'itself', 'mustn', 'those', 'until', 'you', 'hers', 'be', 'than', "shouldn't", "you'll", 'few', "hadn't", 'again', "should've", 'didn', 'below', 'yours', 'no', 'where', 'd', 'once', 'is', 'while', 'yourself', 'y', 'each', 'just', "don't", "she's", 'in', "you've", 'off', 'on', 'so', 'own', 'here', 'too', "you're", "wasn't", 'down', 'don', 'its', 'he', 'above', 'him', 'of', 'hadn', 'they', 'between', 'from', 'doesn', 'there', 'them', 'mightn', 'me', 'about', '

In [5]:
import numpy as np
import nltk
import pandas as pd
import re

from bs4 import BeautifulSoup

def preprocessing(text):
    clean_data = []
    for x in (text[:][0]):
        clean_data =  BeautifulSoup(x, 'html.parser').get_text()
        clean_data = clean_data.lower()#to lower
        clean_data = re.sub(r'[^\w\s]', '', clean_data) # remove punctuation
        #clean_data = nltk.clean_html(clean_data)=>NotImplementedError: To remove HTML markup, use BeautifulSoup's get_text() function
    return clean_data


def preprocessingdf(text):
    #print(text)
    #clean_data = []
    clean_data = text
    clean_data = clean_data.lower()#to lower
    clean_data = BeautifulSoup(clean_data, 'html.parser').get_text()
    return clean_data

test = ['You<p> like &nbsp; to eat apples. He has eaten many apples because he likes eating.']
test_pd = pd.DataFrame(data = test) #makes this into a panda data frame
clean_test = preprocessing(test_pd) #removes punctuation, see above
clean_test

'you like \xa0 to eat apples he has eaten many apples because he likes eating'

In [6]:
data_question['body'][1:9]

1    <p>I accidentally committed the wrong files to...
2    <p>I want to delete a branch both locally and ...
3    <blockquote>\n  <p><strong>Moderator Note:</st...
4    <p>I've been messing around with <a href="http...
5    <p>What is the use of the <code>yield</code> k...
6    <p>After reading <a href="http://groups.google...
7    <p>I mistakenly added files to git using the c...
8    <p>I don't want to rename a remote branch, as ...
Name: body, dtype: object

In [7]:
data_question['body'][1:9].apply(preprocessingdf)
#data_question['clean_body'] = data_question['body'].apply(lambda x : preprocessing(x))


1    i accidentally committed the wrong files to gi...
2    i want to delete a branch both locally and rem...
3    \nmoderator note: given that this question has...
4    i've been messing around with json for some ti...
5    what is the use of the yield keyword in python...
6    after reading hidden features and dark corners...
7    i mistakenly added files to git using the comm...
8    i don't want to rename a remote branch, as des...
Name: body, dtype: object

In [8]:
#data_preprocessed = data_question['body'][1:9].apply(tagText.preprocessing)
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
print(data_preprocessed)

0        piece c code shows peculiar behavior strange r...
1        accidentally committed wrong files git pushed ...
2        want delete branch locally remotely failed att...
3        moderator note given question already sixtysev...
4        ive messing around json time pushing text hurt...
5        use yield keyword python do example im trying ...
6        reading hidden features dark corners cstl comp...
7        mistakenly added files git using command git a...
8        want rename remote branch described rename mas...
9        redirect user one page another using jquery pu...
10       programming language books explain value types...
11       wrote wrong thing commit message change messag...
12       would explain javascript closures someone know...
13       usually would expect stringcontains method see...
14       possible toggle visibility element using funct...
15       array numbers im using push method add element...
16       recently ran javascript code crockfords jslint.

In [9]:
tagText.cleanIngred('i accidentally ')

'accidentally'

In [10]:
data_question.head()

,body,title,tags,SelectedTags,Name
0,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo>,"git,version-control,git-commit,",Question
2,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><git-branch><git-remote>,"git,git-branch,",Question
3,<blockquote>\n <p><strong>Moderator Note:</st...,What is the difference between 'git pull' and ...,<git><git-pull><git-fetch>,"git,",Question
4,"<p>I've been messing around with <a href=""http...",What is the correct JSON content type?,<json><http-headers><content-type>,"json,http-headers,content-type,",Question


In [11]:
data_question.iteritems()

<generator object DataFrame.iteritems at 0x000001BFC171AE08>

In [12]:
data_preprocessed.describe()

count                                                 50000
unique                                                50000
top       debugging cuda code checking compatibilities n...
freq                                                      1
Name: body, dtype: object

In [13]:
import nltk
from collections import defaultdict
    
corpora = defaultdict(list)
        
tokenizer = nltk.RegexpTokenizer(r'\w+')
for id, row in data_preprocessed.iteritems():
    for word in tokenizer.tokenize(row):
        if not word in corpora :
            corpora[word] = 0
        #print(word)
        corpora[word] += 1
    

#print(corpora)
#print(sorted(corpora.items(), reverse=True,  key=lambda x: x[1]))

In [14]:
import matplotlib.pyplot as plt
minimum_count = 200
def count_word_occurencies(df):
        corpora = defaultdict(list)                
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        for id, row in df.iteritems():
            for word in tokenizer.tokenize(row):
                if not word in corpora :
                    corpora[word] = 0
                #print(word)
                corpora[word] += 1
        corpora = dict((k, v) for k, v in corpora.items() if ((v > minimum_count)))
        sorted(corpora)
        max_occurence = max(corpora.values())
        min_occurence = (corpora.values())
        min_occurence = min(corpora.values())
        quartil1 =  np.percentile(list(corpora.values()), 25)
        quartil3 =  np.percentile(list(corpora.values()), 75)
        inter_quartil_range = quartil3 - quartil1
        lower_bound = quartil1 -(1.5 * inter_quartil_range) 
        upper_bound = quartil3 +(1.5 * inter_quartil_range) 
        print (min_occurence)
        print (max_occurence)
        print (quartil1)
        print (quartil3)
        print (inter_quartil_range)
        print (lower_bound)
        print (upper_bound)
        #plt.boxplot(corpora.items())
        #print(corpora.values())
        #quartil1 = np.percentile(corpora.values(),25)
        corpora = dict((k, v) for k, v in corpora.items() if ((v > 1)))# and (v > lower_bound) and (v < upper_bound)))
        return (sorted(corpora.items(), reverse=True,  key=lambda x: x[1]))

dico = count_word_occurencies(data_preprocessed)


In [15]:
dico = count_word_occurencies(data_preprocessed)

201
20052
296.0
1041.5
745.5
-822.25
2159.75


In [16]:
len(dico)

1864

In [17]:
dico

[('like', 20052),
 ('using', 18156),
 ('im', 17409),
 ('use', 16357),
 ('code', 15098),
 ('want', 14713),
 ('get', 14451),
 ('way', 14095),
 ('would', 13548),
 ('file', 13163),
 ('new', 12636),
 ('error', 11625),
 ('this', 11259),
 ('one', 11081),
 ('string', 10799),
 ('1', 10739),
 ('class', 10381),
 ('public', 9653),
 ('function', 9527),
 ('return', 9350),
 ('know', 8714),
 ('need', 8502),
 ('ive', 7792),
 ('0', 7790),
 ('following', 7762),
 ('something', 7598),
 ('example', 7593),
 ('it', 7412),
 ('2', 7389),
 ('set', 7275),
 ('c', 7081),
 ('data', 7061),
 ('work', 6996),
 ('id', 6952),
 ('method', 6896),
 ('value', 6598),
 ('var', 6571),
 ('find', 6264),
 ('name', 6189),
 ('trying', 6110),
 ('also', 6085),
 ('div', 5995),
 ('int', 5980),
 ('object', 5880),
 ('could', 5785),
 ('tried', 5660),
 ('question', 5639),
 ('line', 5498),
 ('git', 5486),
 ('time', 5468),
 ('type', 5426),
 ('files', 5418),
 ('make', 5360),
 ('run', 5257),
 ('see', 5207),
 ('problem', 5176),
 ('list', 5157),
 

In [18]:
print(tagText.count_word_occurencies(data_preprocessed))

[('like', 20052), ('using', 18156), ('im', 17409), ('use', 16357), ('code', 15098), ('want', 14713), ('get', 14451), ('way', 14095), ('would', 13548), ('file', 13163), ('new', 12636), ('error', 11625), ('this', 11259), ('one', 11081), ('string', 10799), ('1', 10739), ('class', 10381), ('public', 9653), ('function', 9527), ('return', 9350), ('know', 8714), ('need', 8502), ('ive', 7792), ('0', 7790), ('following', 7762), ('something', 7598), ('example', 7593), ('it', 7412), ('2', 7389), ('set', 7275), ('c', 7081), ('data', 7061), ('work', 6996), ('id', 6952), ('method', 6896), ('value', 6598), ('var', 6571), ('find', 6264), ('name', 6189), ('trying', 6110), ('also', 6085), ('div', 5995), ('int', 5980), ('object', 5880), ('could', 5785), ('tried', 5660), ('question', 5639), ('line', 5498), ('git', 5486), ('time', 5468), ('type', 5426), ('files', 5418), ('make', 5360), ('run', 5257), ('see', 5207), ('problem', 5176), ('list', 5157), ('user', 5032), ('project', 4969), ('possible', 4959), ('

In [19]:
dico

[('like', 20052),
 ('using', 18156),
 ('im', 17409),
 ('use', 16357),
 ('code', 15098),
 ('want', 14713),
 ('get', 14451),
 ('way', 14095),
 ('would', 13548),
 ('file', 13163),
 ('new', 12636),
 ('error', 11625),
 ('this', 11259),
 ('one', 11081),
 ('string', 10799),
 ('1', 10739),
 ('class', 10381),
 ('public', 9653),
 ('function', 9527),
 ('return', 9350),
 ('know', 8714),
 ('need', 8502),
 ('ive', 7792),
 ('0', 7790),
 ('following', 7762),
 ('something', 7598),
 ('example', 7593),
 ('it', 7412),
 ('2', 7389),
 ('set', 7275),
 ('c', 7081),
 ('data', 7061),
 ('work', 6996),
 ('id', 6952),
 ('method', 6896),
 ('value', 6598),
 ('var', 6571),
 ('find', 6264),
 ('name', 6189),
 ('trying', 6110),
 ('also', 6085),
 ('div', 5995),
 ('int', 5980),
 ('object', 5880),
 ('could', 5785),
 ('tried', 5660),
 ('question', 5639),
 ('line', 5498),
 ('git', 5486),
 ('time', 5468),
 ('type', 5426),
 ('files', 5418),
 ('make', 5360),
 ('run', 5257),
 ('see', 5207),
 ('problem', 5176),
 ('list', 5157),
 

In [20]:
type(dico)

list

In [21]:
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
word_occurence_count = tagText.count_word_occurencies(data_preprocessed)
word_occurence_count_df = pd.DataFrame.from_dict(word_occurence_count).describe()

In [22]:
word_occurence_count_df.head(100)

,1
count,75991.000000
mean,34.263321
std,315.279064
min,2.000000
25%,2.000000
50%,3.000000
75%,6.000000
max,20052.000000


In [23]:
word_occurence_count_df.plot(kind='bar', color="#f56900", title='occurence of word')

In [24]:
dict_tag = count_word_occurencies(data_preprocessed)
dict_tag = str(dict_tag)
dict_tag

201
20052
296.0
1041.5
745.5
-822.25
2159.75


"[('like', 20052), ('using', 18156), ('im', 17409), ('use', 16357), ('code', 15098), ('want', 14713), ('get', 14451), ('way', 14095), ('would', 13548), ('file', 13163), ('new', 12636), ('error', 11625), ('this', 11259), ('one', 11081), ('string', 10799), ('1', 10739), ('class', 10381), ('public', 9653), ('function', 9527), ('return', 9350), ('know', 8714), ('need', 8502), ('ive', 7792), ('0', 7790), ('following', 7762), ('something', 7598), ('example', 7593), ('it', 7412), ('2', 7389), ('set', 7275), ('c', 7081), ('data', 7061), ('work', 6996), ('id', 6952), ('method', 6896), ('value', 6598), ('var', 6571), ('find', 6264), ('name', 6189), ('trying', 6110), ('also', 6085), ('div', 5995), ('int', 5980), ('object', 5880), ('could', 5785), ('tried', 5660), ('question', 5639), ('line', 5498), ('git', 5486), ('time', 5468), ('type', 5426), ('files', 5418), ('make', 5360), ('run', 5257), ('see', 5207), ('problem', 5176), ('list', 5157), ('user', 5032), ('project', 4969), ('possible', 4959), (

In [25]:
tagText.unsupervised_tag(dict_tag, "like git branche totot but java is too good for the dance, test", 2)

['like', 'git']

In [26]:
def unsupervised_tag(dict_word_key, new_question, number_max_tag):
    tags = [word for word in new_question.split() if (word in dict_word_key)]
    return tags[0:number_max_tag]

print(unsupervised_tag(dict_tag, "like git branche totot, test", 20))
print(unsupervised_tag(dict_tag, "like git branche totot, test", 2))
#for word in "like git branche totot, test".split() if (word in dict_word_key)

['like', 'git', 'branche', 'test']
['like', 'git']


In [27]:
for word in "like git branche totot, test".split() :
    print(word)
    if word in str(dict_tag) :
        print("!!!Yes!!!")

like
!!!Yes!!!
git
!!!Yes!!!
branche
!!!Yes!!!
totot,
test
!!!Yes!!!


In [28]:
print(nltk.word_tokenize(data_question['body'][0])[0:10])
print(nltk.tokenize.WhitespaceTokenizer().tokenize(data_question['body'][0])[0:10])
# Tokenize using Punctuations
print(nltk.tokenize.WordPunctTokenizer().tokenize(data_question['body'][0])[0:10])
print(nltk.tokenize.TreebankWordTokenizer().tokenize(data_question['body'][0])[0:10])

['<', 'p', '>', 'Here', 'is', 'a', 'piece', 'of', 'C++', 'code']
['<p>Here', 'is', 'a', 'piece', 'of', 'C++', 'code', 'that', 'shows', 'some']
['<', 'p', '>', 'Here', 'is', 'a', 'piece', 'of', 'C', '++']
['<', 'p', '>', 'Here', 'is', 'a', 'piece', 'of', 'C++', 'code']


In [29]:

from string import punctuation as ponctuation
ponctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [30]:
data_preprocessed[0:9].unique()

array([ 'piece c code shows peculiar behavior strange reason sorting data miraculously makes code almost six times faster include algorithm include ctime include iostream int main   generate data const unsigned arraysize  32768 int dataarraysize unsigned c  0 c  arraysize c datac  stdrand  256   this next loop runs faster stdsortdata data  arraysize  test clockt start  clock long long sum  0 unsigned  0  100000 i   primary loop unsigned c  0 c  arraysize c  datac  128 sum  datac   double elapsedtime  staticcastdoubleclock  start  clockspersec stdcout  elapsedtime  stdendl stdcout  sum    sum  stdendl  without stdsortdata data  arraysize code runs 1154 seconds sorted data code runs 193 seconds initially thought might language compiler anomaly tried java import javautilarrays import javautilrandom public class main  public static void mainstring args   generate data int arraysize  32768 int data  new intarraysize random rnd  new random0 int c  0 c  arraysize c datac  rndnextint  256   th

In [31]:
data_preprocessed.unique()

array([ 'piece c code shows peculiar behavior strange reason sorting data miraculously makes code almost six times faster include algorithm include ctime include iostream int main   generate data const unsigned arraysize  32768 int dataarraysize unsigned c  0 c  arraysize c datac  stdrand  256   this next loop runs faster stdsortdata data  arraysize  test clockt start  clock long long sum  0 unsigned  0  100000 i   primary loop unsigned c  0 c  arraysize c  datac  128 sum  datac   double elapsedtime  staticcastdoubleclock  start  clockspersec stdcout  elapsedtime  stdendl stdcout  sum    sum  stdendl  without stdsortdata data  arraysize code runs 1154 seconds sorted data code runs 193 seconds initially thought might language compiler anomaly tried java import javautilarrays import javautilrandom public class main  public static void mainstring args   generate data int arraysize  32768 int data  new intarraysize random rnd  new random0 int c  0 c  arraysize c datac  rndnextint  256   th

In [32]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

stemmer = nltk.stem.porter.PorterStemmer()
stopword = nltk.corpus.stopwords.words('english')

#X = data_preprocessed.unique()
X = data_preprocessed[0:9].unique()

def stem_and_filter(doc):
    tokens = [stemmer.stem(w) for w in analyzer(doc)]
    return [token for token in tokens if token.isalpha()]

analyzer = TfidfVectorizer().build_analyzer()
cv = TfidfVectorizer(lowercase=True, stop_words="english", analyzer=stem_and_filter, min_df=0.00, max_df=0.3)  # we remove words if it appears in more than 30 % of the corpus (not found stopwords like Box, Christmas and so on)
word_count_vector = cv.fit_transform(X)
print("TF_IDF_matrix :", word_count_vector.shape, "of", word_count_vector.dtype)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["tf_idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['tf_idf_weights'])


TF_IDF_matrix : (9, 256) of float64


,tf_idf_weights
file,2.203973
tri,2.203973
remot,2.203973
branch,2.203973
differ,2.203973
start,2.203973
thi,2.203973
delet,2.203973
piec,2.203973
return,2.203973


In [33]:
tf = word_count_vector

from sklearn.decomposition import LatentDirichletAllocation
no_topics = 20
'''
tf_vectorizer = COuntVectorizer(max_df)
tf = tf_vectorizer.fit_transform(documents)
'''
# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)


def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic :", topic_idx, ":" )
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 5
display_topics(lda, cv.get_feature_names(), no_top_words)
# reprendre forcement avec countVectorizer

Topic : 0 :
reason applicationxjavascript mani keyword pull
Topic : 1 :
branch primari anomali method stop
Topic : 2 :
anomali wrong start stdendl branch
Topic : 3 :
faster method subsequ browser selfleftchild
Topic : 4 :
moder sixtyseven differ getchildcandid target
Topic : 5 :
remot case show fail sinc
Topic : 6 :
static self uptod staticcastdoubleclock random
Topic : 7 :
anoth modul uptod might complangcmoder
Topic : 8 :
algorithm thing python unsort remotesoriginbugfix
Topic : 9 :
rest server unsort whether api
Topic : 10 :
follow them systemoutprintlnsum describ goe
Topic : 11 :
from seen them exampl strang
Topic : 12 :
int here systemoutprintlnsum commit loop
Topic : 13 :
distanc seen getchildcandidatesself written silli
Topic : 14 :
textxjavascript gather make mistakenli mani
Topic : 15 :
systemnanotim sinc branch case given
Topic : 16 :
again text primari success call
Topic : 17 :
rest method remot renam similar
Topic : 18 :
need support accident rnd around
Topic : 19 :
branch 

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation
no_topics = 20
documents = data_preprocessed[0:9].unique()

tf_vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2, 
    max_features=1000,
    stop_words='english'
)
tf = tf_vectorizer.fit_transform(documents)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

'''
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic :", topic_idx, ":" )
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
'''

no_top_words = 5
tagText.display_topics(lda, tf_vectorizer.get_feature_names(), no_top_words)
print(tf_vectorizer.get_feature_names())
print(lda)


Topic : 0 :
branch remote local git pushed
Topic : 1 :
id question start similar returned
Topic : 2 :
pushed git commit local new
Topic : 3 :
start commit pushed branch piece
Topic : 4 :
returned similar want commit git
Topic : 5 :
local git start want branch
Topic : 6 :
undo remote commit deleted include
Topic : 7 :
git branch commit new remote
Topic : 8 :
main git id local branch
Topic : 9 :
question start code similar include
Topic : 10 :
int id local main include
Topic : 11 :
main undo question returned similar
Topic : 12 :
result piece code files returned
Topic : 13 :
pushed code local new branch
Topic : 14 :
int code include start main
Topic : 15 :
pushed question result branch deleted
Topic : 16 :
result pull new include want
Topic : 17 :
similar commit branch deleted question
Topic : 18 :
include want piece int deleted
Topic : 19 :
start new int result pull
['branch', 'code', 'commit', 'deleted', 'files', 'git', 'id', 'include', 'int', 'local', 'main', 'new', 'piece', 'pull', '

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation
no_topics = 20
documents = data_preprocessed.unique()

tf_vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2, 
    max_features=1000,
    stop_words='english'
)
tf = tf_vectorizer.fit_transform(documents)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

'''
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic :", topic_idx, ":" )
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
'''

no_top_words = 5
tagText.display_topics(lda, tf_vectorizer.get_feature_names(), no_top_words)
print(tf_vectorizer.get_feature_names())
print(lda)


Topic : 0 :
files add directory path folder
Topic : 1 :
like way want know use
Topic : 2 :
error project version run tried
Topic : 3 :
file python line import command
Topic : 4 :
use understand used example thread
Topic : 5 :
use image difference php variable
Topic : 6 :
git changes branch commit repository
Topic : 7 :
exception code error library try
Topic : 8 :
object array value true false
Topic : 9 :
div field content width height
Topic : 10 :
im using ive code time
Topic : 11 :
string script date java output
Topic : 12 :
data table database select query
Topic : 13 :
function var return code foo
Topic : 14 :
server user request api json
Topic : 15 :
text view android line size
Topic : 16 :
page input button form javascript
Topic : 17 :
application app web url ios
Topic : 18 :
class public int void new
Topic : 19 :
test set tests characters apply
['00', '10', '100', '1000', '11', '12', '13', '15', '16', '20', '2008', '30', '40', '50', 'abc', 'able', 'abstract', 'accept', 'access', '

In [36]:
from joblib import dump, load
dump(lda, 'lda.joblib') 
ldaLoad = load('lda.joblib') 

dump(lda, 'lda.joblib') 
ldaLoad = load('lda.joblib') 

print(ldaLoad)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)


In [37]:
tagText.lda_prepare_tag(data_preprocessed)

(LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
              evaluate_every=-1, learning_decay=0.7,
              learning_method='online', learning_offset=50.0,
              max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
              n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
              topic_word_prior=None, total_samples=1000000.0, verbose=0),
            branch      code    commit   deleted     files       git        id  \
 Topic0   2.518349  0.766361  1.105769  0.668466  0.954405  1.540644  0.793530   
 Topic1   0.638906  0.700683  0.705706  0.626246  0.721678  0.674028  1.314006   
 Topic2   0.771245  0.631296  0.875226  0.724025  0.757603  0.878648  0.763108   
 Topic3   0.837393  0.705912  0.879399  0.682625  0.776757  0.691887  0.787103   
 Topic4   0.666912  0.630740  0.805768  0.760319  0.664611  0.787494  0.680324   
 Topic5   0.785500  0.735221  0.684820  0.757452  0.729894  0.804946  0.760208   
 Topic6   0.727398  0.715621  0

In [38]:
topicnames = ["Topic" + str(i) for i in range(lda.n_topics)]
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda.components_)

# Assign Column and Index
df_topic_keywords.columns = tf_vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

def predict_topic_LDA(text):#, nlp=nlp):
    global sent_to_words
    '''global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)
    '''
    mytext_4 = tf_vectorizer.transform(text)
    print(mytext_4)
    # Step 4: LDA Transform
    topic_probability_scores = lda.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

#mytext = ["git","it","is","so","good"]
mytext = ["git it is so good"]
topic, prob_scores = predict_topic_LDA(text = mytext)
topic_array = np.array(topic)
'''
print(topic)
print(topic_array)
print(prob_scores)
'''
feature_names_lda = tf_vectorizer.get_feature_names()
#topic_dict = dict(topic)
print (" ".join([feature_names_lda[i]
                for i in topic_array.argsort()[:-no_top_words - 1:-1]]))
"""    tags = [word for word in new_question.split() if (word in dict_word_key)]
"""

  (0, 393)	1
  (0, 400)	1
git changes branch commit repository


'    tags = [word for word in new_question.split() if (word in dict_word_key)]\n'

In [39]:
lda_tagText, df_topic_keywords_tagText, tf_vectorizer = tagText.lda_prepare_tag(data_preprocessed)
            
tagText.lda_predict('git est un outils de code source. C\'est bien', lda_tagText, df_topic_keywords, tf_vectorizer, 5)

IndexError: list index out of range

In [ ]:
data_preprocessed

In [ ]:
lda_tagText, df_topic_keywords_tagText, tf_vectorizer = tagText.lda_prepare_tag_and_save(data_preprocessed)
            
tagText.lda_predict('git est un outils de code source. C\'est bien', lda_tagText, df_topic_keywords, tf_vectorizer, 5)

In [ ]:
from joblib import dump, load
--> 206         self.joblib.dump(lda, 'lda.joblib',0)
    207         self.joblib.dump(df_topic_keywords, 'df_lda_topic_keywords.joblib',0)
    208         self.joblib.dump(tf_vectorizer, 'tf_lda_vectorizer.joblib',0)

In [ ]:
'''
dump(lda_tagText, 'lda.joblib',0)
dump(df_topic_keywords, 'df_lda_topic_keywords.joblib')
dump(tf_vectorizer, 'tf_lda_vectorizer.joblib')
'''

In [ ]:
nmf_tagText, df_topic_keywords_tagText, tf_vectorizer = tagText.nmf_prepare_tag(data_preprocessed)
            
nmf_predict(text, nmf_tagText, df_topic_keywords, tfidf_vectorizer)

In [ ]:

unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

In [ ]:
def tag_lda(model, post, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        lst3 = [value for value in post if value in topic] 
        '''print (" ".join(post[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]))
'''
tag_lda(lda, ["git", "branch", "test", "around"], no_top_words)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
no_features=0
X = data_preprocessed[0:9].unique()

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95,
    min_df=2,
    max_features=None,
    stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(X)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

no_topics = 5

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)

In [ ]:
from joblib import dump, load
dump(nmf, 'nmf.joblib') 
nmfLoad = load('nmf.joblib') 
print(nmfLoad)

In [ ]:
print(tfidf_feature_names)

In [ ]:
def predict_topicNMF(text):#, nlp=nlp):
    global sent_to_words
    mytext_4 = tfidf_vectorizer.transform(text)
    print(mytext_4)
    # Step 4: LDA Transform
    topic_probability_scores = nmf.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

#mytext = ["git","it","is","so","good"]
mytext = ["git it is so good"]
topic, prob_scores = predict_topicNMF(text = mytext)
topic_array = np.array(topic)
feature_names_nmf = tfidf_vectorizer.get_feature_names()
#topic_dict = dict(topic)
print (" ".join([feature_names_nmf[i]
                for i in topic_array.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
lda_tagText, df_lda_topic_keywords, tf_lda_vectorizer = tagText.lda_prepare_tag_and_save(data_preprocessed)
mnf_tagText, df_nmf_topic_keywords, tf_mnf_vectorizer = tagText.nmf_prepare_tag_and_save(data_preprocessed)

tagText.lda_predict('git est un outils de code source. C\'est bien', lda_tagText, df_lda_topic_keywords, tf_lda_vectorizer, 5)
tagText.nmf_predict('git est un outils de code source. C\'est bien', mnf_tagText, df_nmf_topic_keywords, tf_mnf_vectorizer, 5)


In [ ]:
nmf

In [ ]:

import numpy as np
import nltk
import pandas as pd
import os
import sys

sys.path.insert(0, './Python/')

from tagText import TagText


tagText = TagText()
print(tagText.urlDirectory)
print(tagText.fileName)


def supervisedTagPurposer():
    return tagPurposer('supervised', 'This is a test to have tag GIT & branche', 'false', 'false')

def supervisedTagPurposerAndReload():
    return tagPurposer('supervised', 'This is a test to have tag GIT & branche', 'true', 'false')

def supervisedTagPurposerAndReloadAndSave():
    return tagPurposer('supervised', 'This is a test to have tag GIT & branche', 'true', 'true')

def tagLDAPurposer():
    return tagPurposer('LDA', 'This is a test to have tag GIT & branche', 'false', 'false')

def tagLDAPurposerAndReload():
    return tagPurposer('LDA', 'This is a test to have tag GIT & branche', 'true', 'false')

def tagLDAPurposerAndReloadAndSave():
    return tagPurposer('LDA', 'This is a test to have tag GIT & branche', 'true', 'true')

def tagNMFPurposer():
    return tagPurposer('NMF', 'This is a test to have tag GIT & branche', 'false', 'false')

def tagNMFPurposerAndReload():
    return tagPurposer('NMF', 'This is a test to have tag GIT & branche', 'true', 'false')

def tagNMFPurposerAndReloadAndSave():
    return tagPurposer('NMF', 'This is a test to have tag GIT & branche', 'true', 'true')

def tagPurposer(mode, message, reload, saveModel):
    if mode.lower() == 'supervised':
        return superviedTagPurposerFromMessage(message, reload.lower(), saveModel.lower())
    if mode.lower() == 'lda' or mode.lower() =='nmf':
        return unsuperviedTagTFIDFPurposerFromMessage(mode.lower(), message, reload.lower(), saveModel.lower())
    return '/i\\Mode indefined/i\\'

def superviedTagPurposerFromMessage(message, reload, saveModel):
    tagText = TagText()
    tags = ''
    if reload.lower() == 'true':
        data_question = tagText.read_source()
        data_preprocessed = data_question['body'].apply(tagText.preprocessing)
        logistic_regression = tagText.supervised_prepare_tag(data_preprocessed)
        tags = tagText.supervised_predict(message, logistic_regression, 5)
    else:
        logistic_regression = tagText.supervised_prepare_tag_load()
        tags = tagText.supervised_predict(message, logistic_regression, 5)
    return tags

def unsuperviedTagTFIDFPurposerFromMessage(option, message, reloadModel, saveModel):
    tagText = TagText()
    tags = ''
    if option.lower() != 'lda' and option.lower() != 'nmf':
        return '/i\\Mode indefined/i\\'
    if reloadModel.lower() == 'true':
        data_question = tagText.read_source()
        data_preprocessed = data_question['body'].apply(tagText.preprocessing)
        if option.lower() == 'lda':
            if saveModel.lower() == 'true':
                lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.lda_prepare_tag_and_save(data_preprocessed)
                tags = tagText.lda_predict(message, lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
            else:
                lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.lda_prepare_tag(data_preprocessed)
                tags = tagText.lda_predict(message, lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
        if option.lower() == 'nmf':
            if saveModel.lower() == 'true':
                nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.nmf_prepare_tag_and_save(data_preprocessed)
                tags = tagText.nmf_predict(message, nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
            else:
                nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.nmf_prepare_tag(data_preprocessed)
                tags = tagText.nmf_predict(message, nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
    else:
        if option.lower() == 'lda':
            lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.lda_prepare_tag_load()
            tags = tagText.lda_predict(message, lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
        if option.lower() == 'nmf':
            nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.nmf_prepare_tag_load()
            tags = tagText.nmf_predict(message, nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
    return tags


In [ ]:
tagLDAPurposerAndReloadAndSave()

In [ ]:
#supervised

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

stemmer = nltk.stem.porter.PorterStemmer()
stopword = nltk.corpus.stopwords.words('english')

#X = data_preprocessed.unique()
 data_question['SelectedTags']
X = data_preprocessed[0:9].unique()

def stem_and_filter(doc):
    tokens = [stemmer.stem(w) for w in analyzer(doc)]
    return [token for token in tokens if token.isalpha()]

analyzer = TfidfVectorizer().build_analyzer()
cv = TfidfVectorizer(lowercase=True, stop_words="english", analyzer=stem_and_filter, min_df=0.00, max_df=0.3)  # we remove words if it appears in more than 30 % of the corpus (not found stopwords like Box, Christmas and so on)
word_count_vector = cv.fit_transform(X)
print("TF_IDF_matrix :", word_count_vector.shape, "of", word_count_vector.dtype)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["tf_idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['tf_idf_weights'])

In [43]:
 data_preprocessed[0:9]

0    piece c code shows peculiar behavior strange r...
1    accidentally committed wrong files git pushed ...
2    want delete branch locally remotely failed att...
3    moderator note given question already sixtysev...
4    ive messing around json time pushing text hurt...
5    use yield keyword python do example im trying ...
6    reading hidden features dark corners cstl comp...
7    mistakenly added files git using command git a...
8    want rename remote branch described rename mas...
Name: body, dtype: object

In [42]:
data_question[0:9]['SelectedTags']

0            java,c++,performance,optimization,
1               git,version-control,git-commit,
2                               git,git-branch,
3                                          git,
4               json,http-headers,content-type,
5    python,iterator,generator,yield,coroutine,
6                c++,operators,code-formatting,
7               git,version-control,git-commit,
8                               git,git-branch,
Name: SelectedTags, dtype: object

In [68]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


X_train = np.array(["new york is a hell of a town",
                "new york was originally dutch",
                "the big apple is great",
                "new york is also called the big apple",
                "nyc is nice",
                "people abbreviate new york city as nyc",
                "the capital of great britain is london",
                "london is in the uk",
                "london is in england",
                "london is in great britain",
                "it rains a lot in london",
                "london hosts the british museum",
                "new york is great and so is london",
                "i like london better than new york",
                "france"])
'''
y_train_text = [["new york"],["new york"],["new york"],["new york"],    
                ["new york"],["new york"],["london"],["london"],         
                ["london"],["london"],["london"],["london"],
                ["new york","England"],["new york"],["France"]]
'''

y_train = ["new york","new york","new york","new york",    
            "new york","new york","london","london",         
            "london","london","london","london",
           "new york,England","new york,paris","France"]
y_train_text = [item.split(',') for item in y_train]
print(y_train_text)

#lb = preprocessing.MultiLabelBinarizer(classes=("new york","london","England","France"))
print(data_question['SelectedTags'][1:9])
list_tag = data_question['SelectedTags'][1:9].tolist()
y_train_text = [item[:-1].split(',') for item in list_tag]

print(y_train_text)
Y = lb.fit_transform(y_train_text)
print(Y)

lb = preprocessing.MultiLabelBinarizer(classes=('git', 'json'))
forest = RandomForestClassifier(n_estimators=10, random_state=1)

classifier = Pipeline([
('vectorizer', CountVectorizer()),
('tfidf', TfidfTransformer()),
#('clf',  DecisionTreeClassifier(random_state=0))
('clf', RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0))
#('clf', OneVsRestClassifier(LinearSVC()))])
])
classifier.fit(X_train, Y)
predicted = classifier.predict_proba(["new york is great and so is london"])
print(predicted)
print([item[0][1] for item in predicted] )


tempTag = [item[0][1] for item in predicted]
print(lb.classes_)
print([i for i, x in enumerate(tempTag) if x > 0.050])
list_id = [i for i, x in enumerate(tempTag) if x > 0.050]
print([id for id in list_id])
print([lb.classes_[id] for id in list_id])
#print("Accuracy Score: ",accuracy_score(Y_test, predicted))

[['new york'], ['new york'], ['new york'], ['new york'], ['new york'], ['new york'], ['london'], ['london'], ['london'], ['london'], ['london'], ['london'], ['new york', 'England'], ['new york', 'paris'], ['France']]
1               git,version-control,git-commit,
2                               git,git-branch,
3                                          git,
4               json,http-headers,content-type,
5    python,iterator,generator,yield,coroutine,
6                c++,operators,code-formatting,
7               git,version-control,git-commit,
8                               git,git-branch,
Name: SelectedTags, dtype: object
[['git', 'version-control', 'git-commit'], ['git', 'git-branch'], ['git'], ['json', 'http-headers', 'content-type'], ['python', 'iterator', 'generator', 'yield', 'coroutine'], ['c++', 'operators', 'code-formatting'], ['git', 'version-control', 'git-commit'], ['git', 'git-branch']]


KeyError: 'git'

In [72]:
for id, row in data_question['SelectedTags'][1:9].iteritems():
            for word in tokenizer.tokenize(row):
                if not word in corpora :
                    corpora[word] = 0
                #print(word)
                corpora[word] += 1

tags = [item[:-1].split(',') for item in list_tag]
for id, row in tags.iteritems():
            for word in tokenizer.tokenize(row):
                if not word in corpora :
                    corpora[word] = 0
                #print(word)
                corpora[word] += 1

AttributeError: 'list' object has no attribute 'iteritems'